<a href="https://colab.research.google.com/github/kmjohnson3/Intro-to-MRI/blob/master/NoteBooks/Recon_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MRI Reconstruction Excercise

This Jupyter notebook provides some hands on experience with raw data from an MRI scan. Each code cell can be run by clicking on the upper left corner. You can also run all by using the "Runtime" menu on the top menu bar. When you modify one of the reconstruction paramaters, think about what you expect the change to be.

# Objectives
*   Reconstruct images from an actual scan. 
*   Understand the core processing steps to reconstruct an image
*   Explore tradeoffs in reconstruction and sampling

In python you need to load libraries to use them. This first cell imports a couple of key libraries to reconstruct images.

In [1]:
# This is comment, Python will ignore this line

# Import libraries (load libraries which provide some functions)
%matplotlib inline
import numpy as np # array library
import math
import cmath
import pickle

# For interactive plotting
from ipywidgets import interact, interactive, FloatSlider, IntSlider
from IPython.display import clear_output, display, HTML

# for plotting modified style for better visualization
import matplotlib.pyplot as plt 
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 4
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['legend.fontsize'] = 16

# Download Raw Data
We are going to download raw data from scans collected on the scanner. These are 2D brain scans collected with clinical sequence paramaters. There are 5 set of scans.

In [2]:
# Get some data - Data is stored a Python pickled object
import os
if not os.path.exists("ReconExercise.tar"):
  !wget https://www.dropbox.com/s/1w1izrohybpwqaw/ReconExercise.tar 
  !tar xvf ReconExercise.tar 

# import the data
scans = []
for scan_number in range(5):
  filename = f'BrainScan{scan_number+1}.p'

  # Load the data
  dbfile = open(filename, 'rb')      
  scan_data = pickle.load(dbfile) 
  dbfile.close()

  # Fix a typo!
  if scan_data['Sequence'] == 'Spoiled Gradent Echo':
    scan_data['Sequence'] = 'Spoiled Gradient Echo'

  scans.append(scan_data)

--2022-01-04 23:43:01--  https://www.dropbox.com/s/1w1izrohybpwqaw/ReconExercise.tar
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/1w1izrohybpwqaw/ReconExercise.tar [following]
--2022-01-04 23:43:01--  https://www.dropbox.com/s/raw/1w1izrohybpwqaw/ReconExercise.tar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc743afcfac5d266f1490e1d2ba4.dl.dropboxusercontent.com/cd/0/inline/BdJy_LQsnFLJ3Qe6DmblVbzn5Ifa5dID0gx1iHNXwbViCEIYmYQBGNts1M3dEybkbZ_PgRzu0C3gJpKPfs5D4DlX0v_tBpHWsDtGei3YNnfYAR7-lHyhF0XgAJoTgcQ7QsCxBu7teOHVqEh1LrM9fvKS/file# [following]
--2022-01-04 23:43:02--  https://uc743afcfac5d266f1490e1d2ba4.dl.dropboxusercontent.com/cd/0/inline/BdJy_LQsnFLJ3Qe6DmblVbzn5Ifa5dID0gx1iHNXwbViCEIYmYQBGNts1M3dEybkbZ_PgRzu0C3

This loads the data. You can change the filename to by changing the series number.

This just plots the raw k-space data. The vendor samples the readout direction at a rate 2x higher than that required to support the desired FOV. Some of the data sets are also padded at the edges of the k-space. Due to the way this was created, specifically a set step to convert a multi-coil experiment to a single coil step, those edges have some small values. 

In [3]:
def plot_kspace(idx):
  scan_data = scans[idx]

  plt.figure(figsize=(15,15))
  plt.imshow(np.log(np.abs(scan_data['kspace'])),cmap='gray')
  plt.grid(False)
  plt.title(f'Full Kspace Scan {scan_number}')
  plt.ylabel(r'$K_x$ [index]')
  plt.xlabel(r'$K_y$ [index]')
  plt.xticks([], [])
  plt.yticks([], [])
  plt.show()

w = interactive(plot_kspace, 
                idx=IntSlider(min=0, max=len(scans)-1, step=1, value=0, description='Scan number'))
display(w)


interactive(children=(IntSlider(value=0, description='Scan number', max=4), Output()), _dom_classes=('widget-i…

**Subsampling of the data**
Below are some subsamplings of the data. They include:

*   Subselecting a set of lines by changing $\Delta k_y $ controlled by accY
*   Subselecting a set of lines by changing $k_{max} $ in x and y controlled by kmaxX, kmaxY
*   Subselecting a set of point by randomly removing a set of points, controlled by random_undersampling_fraction (1=remove none)

**Reconstruction**
This is simple discrete Fourier transform of the data with a Gaussian window function. 
* What is the affect of changing the window function from 1.0 to 0.3 (suggest doing this with kmaxX,kmaxY = 1.0,1.0)


**Suggested changes (all starting with other parameters = 1.0)**
* Run with  $\Delta k_y$ = 1,2,3 see if you can predict what the image looks like from 1?
* Run with kmaxY = 1.0, 0.5,0.1. What happens to resolution and noise? Do any artifacts come up?
* Run with with random undersampling = 1.0 and 0.8. If the appearance less disrupting? 

**Questions**
These are images in magnitude and phase. Some things to look at:
* Is there any correspondance between magnitude and phase? 
* Is the phase uniform across the image?
* What features apear bright and why?
* Is there obervable Gibb's ringing? 


In [4]:
def undersample_data(Kspace, accY, kmaxX, kmaxY, random_undersampling_fraction):
  # Subsampling
  Kspace_us = np.zeros_like(Kspace)

  # Regular - change delta k 
  Kspace_us[:,::accY] = Kspace[:,::accY] 

  # Sub sections - change kmax
  ky,kx = np.meshgrid( np.linspace(-1,1,Kspace.shape[1]),np.linspace(-1,1,Kspace.shape[0]))
  Kspace_us *=  np.abs(kx/kmaxX) < 1
  Kspace_us *=  np.abs(ky/kmaxY) < 1

  # Random undersampling
  mask = np.random.uniform(size=Kspace_us.shape[1:])
  mask = np.expand_dims(mask,axis=0)
  mask = np.repeat(mask,repeats=Kspace_us.shape[0],axis=0)
  Kspace_us *= mask < random_undersampling_fraction

  return Kspace_us 

def inverse_fourier_transform(Kspace_us, window):
  # Window function
  ky,kx = np.meshgrid( np.linspace(-1,1,Kspace_us.shape[1]),np.linspace(-1,1,Kspace_us.shape[0]))
  sigma = window 
  kr = np.sqrt( kx**2 + ky**2)
  k_filter = np.exp( -(kr**2)/(2*sigma**2))

  # This is to do a FFT shift operator so that the center of k-space is at the center of the image
  x,y = np.meshgrid( range(Kspace_us.shape[1]), range(Kspace_us.shape[0]))
  chop = (-1)**( x+y)

  # Fourier Transform with Window Function
  Image = chop*np.fft.fft2(k_filter*Kspace_us*chop)

  return Image


def sample_and_plot(scan_idx, accY, kmaxX, kmaxY, random_undersampling_fraction, window):

  # Grab the scan data
  scan_data = scans[scan_idx]
  Kspace = scan_data['kspace'] 

  # Subsample the data
  Kspace_us = undersample_data(Kspace, accY=accY, kmaxX=kmaxX, kmaxY=kmaxY, random_undersampling_fraction=random_undersampling_fraction)

  # Reconstruct that data
  image = inverse_fourier_transform( Kspace_us, window=window)
  
  
  crop = image.shape[0] // 4

  # Show the subsampled data
  plt.figure(figsize=(20,10))
  
  plt.subplot(131)
  plt.imshow(np.log(1e-7+np.abs(Kspace_us)),cmap='gray')
  plt.grid(False)
  plt.title(f'Subsampled Kspace Scan {scan_number}')
  plt.ylabel(r'$K_x$ [index]')
  plt.xlabel(r'$K_y$ [index]')
  plt.xticks([], [])
  plt.yticks([], [])

  plt.subplot(132)
  plt.imshow(np.abs(image[crop:-crop,:]),cmap='gray')
  plt.grid(False)
  plt.title(f'Reconstructed Image {scan_number}')
  plt.ylabel(r'$x$ [index]')
  plt.xlabel(r'$y$ [index]')
  plt.clim(0, 8*np.mean(np.abs(image)))
  plt.xticks([], [])
  plt.yticks([], [])

  plt.subplot(133)
  plt.imshow(np.angle(image[crop:-crop,:]),cmap='gray')
  plt.grid(False)
  plt.title(f'Reconstructed Phase {scan_number}')
  plt.ylabel(r'$x$ [index]')
  plt.xlabel(r'$y$ [index]')
  plt.xticks([], [])
  plt.yticks([], [])

  plt.show()


w = interactive(sample_and_plot, 
                scan_idx=IntSlider(min=0, max=len(scans)-1, step=1, value=0, description='Scan number', continuous_update=False),
                accY=IntSlider(min=1, max=4, step=1, value=1, description='Stride in Y', continuous_update=False),
                kmaxX=FloatSlider(min=0.1, max=1, step=0.1, value=1, description='Kmax X', continuous_update=False),
                kmaxY=FloatSlider(min=0.1, max=1, step=0.1, value=1, description='Kmax Y', continuous_update=False),
                window=FloatSlider(min=0.1, max=1.0, step=0.1, value=1.0, description='Window amount', continuous_update=False),
                random_undersampling_fraction=FloatSlider(min=0.1, max=1, step=0.1, value=1, description='Rand sample', continuous_update=False))
                     
display(w)


interactive(children=(IntSlider(value=0, continuous_update=False, description='Scan number', max=4), IntSlider…

Can you use these images to determine what scan this was? Check your answer by running the below code cell below.

**There are five scans:**

* Spin echo with Inversion Prep
  * TR = 9000ms
  * TE = 81ms

* Spin Echo 
  * TR = 6190ms
  * TE = 113ms

* Spoiled Gradient Echo
  * TR = 250ms
  * TE = 3.4ms
  * Flip angle = 70

* Spoiled Gradient Echo after Contrast
  * TR = 250ms
  * TE = 2.64ms
  * Flip angle = 70

* Spin Echo 
  * TR = 419ms
  * TE = 9.4ms


**It may be useful to know some of the relaxation values in the brain:**
* White Matter - The inner tissue of the brain
  * T1 = 1000 ms
  * T2 = 70 ms

* Gray Matter - The thinner cortical tissue surroundings white matter.  
  * T1 = 1800 ms
  * T2 = 100 ms

* Cerebral Spinal Fluid - A water like fluid surroundings with two large cavities in the center of the brain.
  * T1 = 4000 ms
  * T2 = 800 ms




In [5]:
for idx, scan_data in enumerate(scans):
  print(f'Scan number {idx}')
  print('           Sequence Type = ' + str(scan_data['Sequence']))
  print('           TR = ' + str(scan_data['TR']) + ' ms')
  print('           TE = ' + str(scan_data['TE']) + ' ms')
  if scan_data['Sequence'] == 'Spoiled Gradient Echo':
    print('         Flip = ' + str(scan_data['Flip']) + ' degrees')


Scan number 0
           Sequence Type = Spin Echo
           TR = 9000 ms
           TE = 81 ms
Scan number 1
           Sequence Type = Spoiled Gradient Echo
           TR = 250 ms
           TE = 3.4 ms
         Flip = 70 degrees
Scan number 2
           Sequence Type = Spoiled Gradient Echo
           TR = 250 ms
           TE = 2.64 ms
         Flip = 70 degrees
Scan number 3
           Sequence Type = Spin Echo
           TR = 419 ms
           TE = 9.4 ms
Scan number 4
           Sequence Type = Spin Echo
           TR = 6190 ms
           TE = 113 ms
